# TRELLIS: Text/Image to 3D Asset Generator

Generate high-quality 3D assets from text prompts or images using Microsoft's TRELLIS model.

**Model:** [microsoft/TRELLIS-text-xlarge](https://huggingface.co/microsoft/TRELLIS-text-xlarge)

**Requirements:**
- GPU Runtime (T4, L4, A100 recommended)
- ~16GB+ GPU memory

**Instructions:**
1. Run all setup cells in order (Steps 1.1-1.6)
2. Optionally upload `game_assets_text_to_3d.json` for batch generation
3. Run generation cells
4. Download the output folder when done

---

##  Step 1: Setup - Install Dependencies

This step takes ~5-10 minutes on first run.

In [ ]:
#@title 1.1 Clone TRELLIS Repository
import os
import sys

# Check if already cloned
if not os.path.exists('/content/TRELLIS'):
    !git clone --recurse-submodules https://github.com/gtava5813/TRELLIS.git /content/TRELLIS
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

%cd /content/TRELLIS
sys.path.insert(0, '/content/TRELLIS')

In [ ]:
#@title 1.2 Fix PyTorch/Torchvision Version Mismatch
# This fixes the 'torchvision::nms does not exist' error

import subprocess
import sys

def get_torch_info():
    import torch
    cuda_version = torch.version.cuda
    torch_version = torch.__version__.split('+')[0]
    return torch_version, cuda_version

torch_ver, cuda_ver = get_torch_info()
print(f"Current: PyTorch {torch_ver}, CUDA {cuda_ver}")

# Colab sometimes has mismatched torch/torchvision - fix it
print("\n📦 Ensuring compatible torch/torchvision versions...")
!pip install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu121 -q

# Verify fix
import importlib
import torch
import torchvision
importlib.reload(torch)
importlib.reload(torchvision)
print(f"\n✅ PyTorch {torch.__version__}")
print(f"✅ Torchvision {torchvision.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
#@title 1.3 Install Core Dependencies

# Install basic dependencies
!pip install -q pillow imageio imageio-ffmpeg tqdm easydict \
    opencv-python-headless scipy ninja transformers huggingface-hub safetensors plyfile

# Install 3D processing (use latest open3d, not pinned old version)
!pip install -q trimesh open3d xatlas pyvista pymeshfix igraph

# Install rembg for background removal
!pip install -q "rembg[gpu]" onnxruntime-gpu

# Install utils3d (without deps to avoid open3d version conflicts)
!pip install -q --no-deps git+https://github.com/EasternJournalist/utils3d.git@9a4eb15e4021b67b12c460c7057d642626897ec8

print("\n✅ Core dependencies installed")

In [ ]:
#@title 1.4 Install CUDA Extensions (xformers, spconv, kaolin)
import torch

cuda_version = torch.version.cuda
cuda_major = cuda_version.split('.')[0]
print(f"CUDA version: {cuda_version}")

# Install xformers
print("\n📦 Installing xformers...")
!pip install -q xformers==0.0.27.post2 --index-url https://download.pytorch.org/whl/cu121

# Install spconv (cu120 works with cu121)
print("\n📦 Installing spconv...")
if cuda_major == "12":
    !pip install -q spconv-cu120
else:
    !pip install -q spconv-cu118

# Install kaolin
print("\n📦 Installing kaolin...")
!pip install -q kaolin -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.4.0_cu121.html

print("\n✅ CUDA extensions installed")

In [ ]:
#@title 1.5 Build nvdiffrast and diffoctreerast (takes ~5 min)
import os

ext_dir = "/tmp/trellis_ext"
os.makedirs(ext_dir, exist_ok=True)

# nvdiffrast
print("📦 Building nvdiffrast...")
nvdiff_dir = f"{ext_dir}/nvdiffrast"
if not os.path.exists(nvdiff_dir):
    !git clone https://github.com/NVlabs/nvdiffrast.git {nvdiff_dir}
!pip install -q {nvdiff_dir}

# diffoctreerast  
print("\n📦 Building diffoctreerast...")
diffoctree_dir = f"{ext_dir}/diffoctreerast"
if not os.path.exists(diffoctree_dir):
    !git clone --recurse-submodules https://github.com/JeffreyXiang/diffoctreerast.git {diffoctree_dir}
!pip install -q {diffoctree_dir}

# mip-splatting gaussian rasterization
print("\n📦 Building mip-splatting...")
mipsplat_dir = f"{ext_dir}/mip-splatting"
if not os.path.exists(mipsplat_dir):
    !git clone https://github.com/autonomousvision/mip-splatting.git {mipsplat_dir}
!pip install -q {mipsplat_dir}/submodules/diff-gaussian-rasterization/

print("\n✅ CUDA extensions built")

In [ ]:
#@title 1.6 Verify Installation
import sys
sys.path.insert(0, '/content/TRELLIS')

print("🔍 Verifying installation...\n")

checks = [
    ("PyTorch", "import torch; print(f'  {torch.__version__}')"),
    ("CUDA", "import torch; print(f'  Available: {torch.cuda.is_available()}')"),
    ("xformers", "import xformers; print('  ✅')"),
    ("spconv", "import spconv; print('  ✅')"),
    ("kaolin", "import kaolin; print('  ✅')"),
    ("nvdiffrast", "import nvdiffrast; print('  ✅')"),
    ("transformers", "import transformers; print('  ✅')"),
    ("open3d", "import open3d; print('  ✅')"),
    ("TRELLIS", "from trellis.pipelines import TrellisTextTo3DPipeline; print('  ✅')"),
]

passed = 0
for name, code in checks:
    print(f"{name}:", end="")
    try:
        exec(code)
        passed += 1
    except Exception as e:
        print(f"  ❌ {str(e)[:50]}")

print(f"\n{'='*40}")
print(f"✅ {passed}/{len(checks)} components verified")
if passed >= 7:
    print("🎉 Ready to generate 3D assets!")
else:
    print("⚠️  Some components failed - generation may still work")

---
## Step 2: Load Assets JSON (Optional)

Upload your `game_assets_text_to_3d.json` file for batch generation, or skip to Step 3 for single prompts.

In [ ]:
#@title 2.1 Upload Assets JSON (Optional - for batch generation)
import json
from pathlib import Path

# Upload the JSON file
from google.colab import files
print("Upload game_assets_text_to_3d.json (or skip this cell):")
uploaded = files.upload()

# Load it
json_filename = list(uploaded.keys())[0]
with open(json_filename, 'r') as f:
    data = json.load(f)

assets = data['assets']
theme_prefix = data.get('theme_prefix', '')

print(f"\n✓ Loaded {len(assets)} assets")
print(f"✓ Theme prefix: '{theme_prefix}'")
print(f"\nAssets by category:")

# Count by category
categories = {}
for asset in assets:
    cat = asset['category']
    categories[cat] = categories.get(cat, 0) + 1

for cat, count in sorted(categories.items()):
    print(f"  {cat}: {count} assets")

---
## Step 3: Initialize TRELLIS Pipeline

In [ ]:
#@title 3.1 Load TRELLIS Model
#@markdown Choose your model:
model_type = "Text-to-3D (TRELLIS-text-xlarge)" #@param ["Image-to-3D (TRELLIS-image-large)", "Text-to-3D (TRELLIS-text-xlarge)"]

import os
import sys
sys.path.insert(0, '/content/TRELLIS')

# Set environment variables
os.environ['SPCONV_ALGO'] = 'native'
os.environ['ATTN_BACKEND'] = 'xformers'  # Use xformers instead of flash-attn

print(f"Loading {model_type}...")
print("(This may take 5-10 minutes on first run to download model weights)\n")

if "Image" in model_type:
    from trellis.pipelines import TrellisImageTo3DPipeline
    pipeline = TrellisImageTo3DPipeline.from_pretrained("microsoft/TRELLIS-image-large")
else:
    from trellis.pipelines import TrellisTextTo3DPipeline
    pipeline = TrellisTextTo3DPipeline.from_pretrained("microsoft/TRELLIS-text-xlarge")

pipeline.cuda()
print(f"\n✅ Model loaded and ready!")

---
## Step 4: Generate 3D Assets

In [ ]:
#@title 4.1 Generate Single Asset from Text
#@markdown Enter your text prompt:
prompt = "A futuristic sci-fi chair with neon accents" #@param {type:"string"}
seed = 42 #@param {type:"integer"}
output_name = "generated_asset" #@param {type:"string"}

from trellis.utils import render_utils, postprocessing_utils
import imageio
import os
from pathlib import Path

print(f"\n🎨 Generating 3D from text (seed={seed})...")
print(f"Prompt: {prompt}\n")

outputs = pipeline.run(
    prompt,
    seed=seed,
)

# Create output directory
output_dir = Path("output_models")
output_dir.mkdir(exist_ok=True)

# Export GLB
glb = postprocessing_utils.to_glb(
    outputs['gaussian'][0],
    outputs['mesh'][0],
    simplify=0.9,
    texture_size=1024,
)
glb_path = output_dir / f"{output_name}.glb"
glb.export(str(glb_path))
print(f"✅ GLB saved: {glb_path}")

# Save preview video
video = render_utils.render_video(outputs['mesh'][0])['normal']
video_path = output_dir / f"{output_name}_preview.mp4"
imageio.mimsave(str(video_path), video, fps=30)
print(f"✅ Preview saved: {video_path}")

# Download
from google.colab import files
files.download(str(glb_path))

In [ ]:
#@title 4.2 Test with First Asset from JSON (if uploaded)
import imageio
from pathlib import Path
from trellis.utils import render_utils, postprocessing_utils

def generate_from_text(prompt: str, output_name: str, seed: int = 42, use_theme: bool = True):
    """Generate a 3D model from text prompt."""
    
    # Apply theme prefix for consistent style
    full_prompt = theme_prefix + prompt if use_theme else prompt
    
    print(f"Generating: {output_name}")
    print(f"Prompt: {full_prompt[:80]}...")
    
    # Run TRELLIS text-to-3D
    outputs = pipeline.run(
        full_prompt,
        seed=seed,
    )
    
    # Create output directory
    output_dir = Path("output_models")
    output_dir.mkdir(exist_ok=True)
    
    # Export GLB
    glb = postprocessing_utils.to_glb(
        outputs['gaussian'][0],
        outputs['mesh'][0],
        simplify=0.9,
        texture_size=1024,
    )
    
    glb_path = output_dir / f"{output_name}.glb"
    glb.export(str(glb_path))
    print(f"✓ Saved: {glb_path}")
    
    # Save preview video
    video = render_utils.render_video(outputs['mesh'][0])['normal']
    video_path = output_dir / f"{output_name}_preview.mp4"
    imageio.mimsave(str(video_path), video, fps=30)
    print(f"✓ Preview: {video_path}")
    
    return str(glb_path)

# Test with first asset (if assets were loaded)
if 'assets' in dir() and len(assets) > 0:
    test_asset = assets[0]
    generate_from_text(test_asset['prompt'], test_asset['id'])
else:
    print("⚠️ No assets loaded. Upload JSON in Step 2.1 first, or use cell 4.1 for single prompts.")

---
##  Step 5: Batch Generation

In [ ]:
#@title 5.1 Generate All Assets (Batch)
#@markdown This will generate all assets from the uploaded JSON.
start_from_index = 0 #@param {type:"integer"}
max_assets = 100 #@param {type:"integer"}

from pathlib import Path
import imageio
from trellis.utils import render_utils, postprocessing_utils

if 'assets' not in dir():
    print("❌ No assets loaded. Upload JSON in Step 2.1 first.")
else:
    # Create output directory
    output_dir = Path("output_models")
    output_dir.mkdir(exist_ok=True)
    
    # Track results
    generated = []
    failed = []
    
    assets_to_process = assets[start_from_index:start_from_index + max_assets]
    
    print(f"Starting batch generation of {len(assets_to_process)} assets...\n")
    print("="*60)
    
    for i, asset in enumerate(assets_to_process):
        actual_idx = start_from_index + i
        print(f"\n[{actual_idx + 1}/{len(assets)}] {asset['id']}")
        
        try:
            full_prompt = theme_prefix + asset['prompt']
            
            outputs = pipeline.run(
                full_prompt,
                seed=42,
            )
            
            # Export GLB
            glb = postprocessing_utils.to_glb(
                outputs['gaussian'][0],
                outputs['mesh'][0],
                simplify=0.9,
                texture_size=1024,
            )
            
            glb_path = output_dir / f"{asset['id']}.glb"
            glb.export(str(glb_path))
            print(f"  ✅ Saved: {asset['id']}.glb")
            generated.append(asset['id'])
            
        except Exception as e:
            print(f"  ❌ Failed: {str(e)[:50]}")
            failed.append((asset['id'], str(e)))
    
    print("\n" + "="*60)
    print(f"\n✅ Generated: {len(generated)} assets")
    print(f"❌ Failed: {len(failed)} assets")
    
    if failed:
        print("\nFailed assets:")
        for name, error in failed:
            print(f"  - {name}: {error[:50]}")

---
##  Step 6: Style Consistency (Generate with Same Style)

In [ ]:
#@title 6.1 Generate Multiple Assets with Consistent Style
#@markdown Same seed + CFG settings = consistent style across different prompts

style_seed = 1234 #@param {type:"integer"}
custom_theme_prefix = "low poly cyberpunk, dark moody atmosphere, neon lighting, " #@param {type:"string"}

# Define multiple prompts with same style
style_prompts = [
    "a large building",
    "a medium building", 
    "a small building",
]

from trellis.utils import render_utils, postprocessing_utils
import imageio
import os
import torch
from pathlib import Path

output_dir = Path("output_models/style_batch")
output_dir.mkdir(parents=True, exist_ok=True)

# Style parameters (keep consistent across all generations)
style_params = {
    "seed": style_seed,
    "sparse_structure_sampler_params": {
        "steps": 12,
        "cfg_strength": 7.5,
    },
    "slat_sampler_params": {
        "steps": 12,
        "cfg_strength": 3.0,
    },
}

print(f"🎨 Generating {len(style_prompts)} assets with consistent style...")
print(f"Style seed: {style_seed}")
print(f"Theme: {custom_theme_prefix}\n")

for i, prompt in enumerate(style_prompts):
    full_prompt = custom_theme_prefix + prompt
    print(f"[{i+1}/{len(style_prompts)}] {prompt}")
    
    # Set same seed for consistent style
    torch.manual_seed(style_params["seed"])
    
    outputs = pipeline.run(
        full_prompt,
        seed=style_params["seed"],
        sparse_structure_sampler_params=style_params["sparse_structure_sampler_params"],
        slat_sampler_params=style_params["slat_sampler_params"],
    )
    
    # Export
    safe_name = prompt.replace(" ", "_").replace(",", "")[:30]
    glb = postprocessing_utils.to_glb(
        outputs['gaussian'][0],
        outputs['mesh'][0],
        simplify=0.9,
        texture_size=1024,
    )
    glb.export(str(output_dir / f"{safe_name}.glb"))
    print(f"  ✅ Saved: {safe_name}.glb")

print(f"\n🎉 All {len(style_prompts)} assets generated with consistent style!")

---
## Step 7: Download All Generated Assets

In [ ]:
#@title 7.1 Download All Output Files as ZIP
!cd /content/TRELLIS && zip -r all_outputs.zip output_models/
from google.colab import files
files.download("/content/TRELLIS/all_outputs.zip")